In [18]:
import requests
import pandas as pd
import json
import time
import os
import re

In [19]:
BASE_FILEPATH = '/home/dheyaydesai/Documents/projs/FPL/data'
SEASONS = list(os.walk(BASE_FILEPATH))[0][1]
SAMPLE_MERGED = pd.read_csv(BASE_FILEPATH + '/cleaned_merged_seasons.csv')

SEASONS

/tmp/ipykernel_10044/735153671.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  SAMPLE_MERGED = pd.read_csv(BASE_FILEPATH + '/cleaned_merged_seasons.csv')


['2021-22', '2022-23', '2019-20', '2023-24', '2020-21']

In [20]:
merged_gws_all_szns = None
def merge_files_from_folders(path, seasons):
    all_dataframes = []
    # List of season folders based on the given format
    
    for season in seasons:
        filepath = path + f'/{season}/'
        
        for gw in list(os.listdir(filepath + f'gws')):
            if (gw != 'merged_gw.csv') and  ('xP' not in gw):
                file_path = os.path.join(filepath + f"gws/{gw}")
                gw_number = re.findall(r'\d+', gw)
                # Check if file exists
                if os.path.exists(file_path):
                    try:
                        df = pd.read_csv(file_path, encoding='ISO-8859-1')
                    except UnicodeDecodeError:
                        df = pd.read_csv(file_path, encoding='utf-8', errors='replace')
                    df['gameweek'] = int(gw_number[0])
                    df['season'] = season
                    all_dataframes.append(df)
        
    # Concatenate all dataframes
    combined_df = pd.concat(all_dataframes, ignore_index=True)
    
    return combined_df

merged_gws_all_szns = merge_files_from_folders(BASE_FILEPATH, SEASONS)
merged_gws_all_szns.join(pd.get_dummies(merged_gws_all_szns['position']))

merged_gws_all_szns.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'own_goals', 'penalties_missed', 'penalties_saved',
       'red_cards', 'round', 'saves', 'selected', 'team_a_score',
       'team_h_score', 'threat', 'total_points', 'transfers_balance',
       'transfers_in', 'transfers_out', 'value', 'was_home', 'yellow_cards',
       'gameweek', 'season', 'expected_assists', 'expected_goal_involvements',
       'expected_goals', 'expected_goals_conceded', 'starts'],
      dtype='object')

In [21]:
# Add opponent team name information
df_team = pd.read_csv(BASE_FILEPATH + '/master_team_list.csv')
df_team.rename({'team':'team_drop'}, axis=1,inplace=True)
merged_gws_all_szns = pd.merge(merged_gws_all_szns, df_team[['season', 'team_drop', 'team_name']], 
                     left_on=['season', 'opponent_team'], right_on=['season', 'team_drop'], how='left')
merged_gws_all_szns.rename({'team_name': 'opponent_team_name'}, axis=1, inplace=True)

drop_features = ['xP', 'expected_assists', 'expected_goals', 'expected_goals_conceded', 
                 'expected_goal_involvements', 'kickoff_time', 'team_drop']
for feat in drop_features:
    merged_gws_all_szns.drop(feat, axis=1, inplace=True)

merged_gws_all_szns.rename(columns={'season':'season_x'})
merged_gws_all_szns.columns

Index(['name', 'position', 'team', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'minutes', 'opponent_team', 'own_goals',
       'penalties_missed', 'penalties_saved', 'red_cards', 'round', 'saves',
       'selected', 'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'gameweek', 'season', 'starts',
       'opponent_team_name'],
      dtype='object')

In [22]:
## Merge fixtures data with team name data
def get_modified_fixtures_df():
    all_dfs = []
    for season in SEASONS:
        base_fixtures = f'data/{season}/fixtures.csv'
        base_teams = f'data/{season}/teams.csv'
        df_fixtures = pd.read_csv(base_fixtures).assign(season=season)
        df_teams = pd.read_csv(base_teams)

        # Add home team name and away team name to fixtures
        df_fixtures = df_fixtures.merge(
            df_teams[["code", "id", "name"]],
            left_on="team_h",
            right_on="id",
            suffixes=("", "_home"),
        ).merge(
            df_teams[["code", "id", "name"]],
            left_on="team_a",
            right_on="id",
            suffixes=("", "_away"),
        )
        all_dfs.append(df_fixtures)
    
    all_fixtures = pd.concat(all_dfs, ignore_index=True)
    return pd.concat(
        [
            all_fixtures.rename(
                columns={
                    "team_h": "team",
                    "name": "team_name",
                    "team_h_difficulty": "difficulty",
                    "team_h_score": "score",
                    "team_a": "opponent",
                    "name_away": "opponent_name",
                    "team_a_difficulty": "opponent_difficulty",
                    "team_a_score": "opponent_score",
                }
            ).assign(was_home=True),
            all_fixtures.rename(
                columns={
                    "team_a": "team",
                    "name_away": "team_name",
                    "team_a_difficulty": "difficulty",
                    "team_a_score": "score",
                    "team_h": "opponent",
                    "name": "opponent_name",
                    "team_h_difficulty": "opponent_difficulty",
                    "team_h_score": "opponent_score",
                }
            ).assign(was_home=False),
        ]
    ).reset_index()

ALL_FIXTURES = get_modified_fixtures_df()
# Modify fixtures to match merged_player_gw format
#   Add was home information and create same fixture for home and away team

#ALL_FIXTURES = ALL_FIXTURES[['season', 'id', 'team_a', 'team_h', 'team_a_difficulty', 'team_h_difficulty', 'team_a_score', 'team_h_score']]
ALL_FIXTURES[['season', 'id', 'team_name']], merged_gws_all_szns[['season', 'fixture', 'team']]

ALL_FIXTURES.to_csv('all_fixtures.csv')
merged_gws_all_szns.to_csv('merged_gws_all_szns.csv')

In [23]:
all_player_data = pd.merge(merged_gws_all_szns, 
                                 ALL_FIXTURES[['season', 'id', 'team_name', 'difficulty', 'opponent_difficulty', 'score', 'opponent_score']], 
                                 left_on=['season', 'fixture', 'team'], 
                                 right_on=['season', 'id', 'team_name'], 
                                 how='left')

# Dropping redundant columns
all_player_data.drop(columns=['id', 'team_name'], inplace=True)
all_player_data

,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,was_home,yellow_cards,gameweek,season,starts,opponent_team_name,difficulty,opponent_difficulty,score,opponent_score
0,Ozan Tufan,MID,Watford,0,0,0,0,0.0,557,268,...,False,0,27,2021-22,NaN,Man Utd,4.0,2.0,0.0,0.0
1,Andre Gray,FWD,Watford,0,0,0,0,0.0,383,268,...,False,0,27,2021-22,NaN,Man Utd,4.0,2.0,0.0,0.0
2,Willy Boly,DEF,Wolves,0,0,0,0,0.0,428,270,...,False,0,27,2021-22,NaN,West Ham,3.0,3.0,0.0,1.0
3,Mads Roerslev Rasmussen,DEF,Brentford,0,0,0,0,0.0,89,262,...,True,0,27,2021-22,NaN,Newcastle,2.0,2.0,0.0,2.0
4,Sam Byram,DEF,Norwich,0,0,0,0,0.0,321,269,...,False,0,27,2021-22,NaN,Southampton,3.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101513,Max Meyer,MID,Crystal Palace,0,0,0,0,0.0,143,222,...,False,0,23,2020-21,NaN,Leeds,3.0,2.0,0.0,2.0
101514,Hakim Ziyech,MID,Chelsea,0,0,0,0,0.0,114,226,...,False,0,23,2020-21,NaN,Sheffield Utd,2.0,4.0,2.0,1.0
101515,Jonny Evans,DEF,Leicester,0,1,25,1,1.3,219,228,...,False,1,23,2020-21,NaN,Wolves,3.0,4.0,0.0,0.0
101516,Kyle Edwards,MID,West Brom,0,0,0,0,0.0,421,227,...,False,0,23,2020-21,NaN,Spurs,4.0,2.0,0.0,2.0


In [24]:
all_fixtures_data = pd.read_csv('all_fixtures.csv')

# Filtering for the 2019-20 season and inspecting its structure, since 2019-20 data has discrepencies
fixtures_2019_20_data = all_fixtures_data[all_fixtures_data['season'] == '2019-20']
team_mapping = fixtures_2019_20_data.set_index('id').apply(lambda row: row['team_name'] if row['was_home'] else row['opponent_name'], axis=1).to_dict()
difficulty_mapping = fixtures_2019_20_data.set_index('id')['difficulty'].to_dict()
opponent_difficulty_mapping = fixtures_2019_20_data.set_index('id')['opponent_difficulty'].to_dict()


In [25]:

# Re-defining the mask for the 2019-20 season
mask_2019_20 = all_player_data['season'] == '2019-20'
all_player_data.loc[mask_2019_20, 'team'] = all_player_data.loc[mask_2019_20, 'fixture'].map(team_mapping)
all_player_data.loc[mask_2019_20, 'difficulty'] = all_player_data.loc[mask_2019_20, 'fixture'].map(difficulty_mapping)
all_player_data.loc[mask_2019_20, 'opponent_difficulty'] = all_player_data.loc[mask_2019_20, 'fixture'].map(opponent_difficulty_mapping)

# Checking for missing values in the 2019-20 season data after updating
missing_values_2019_20_after_update = all_player_data[mask_2019_20].isnull().sum()
missing_values_2019_20_after_update[missing_values_2019_20_after_update > 0].sort_values(ascending=False)

position          22560
starts            22560
score             22560
opponent_score    22560
team_a_score         59
team_h_score         59
dtype: int64

In [26]:
# Adjusting the mappings for 'score' and 'opponent_score' based on the 'was_home' column in the main dataset
score_mapping = fixtures_2019_20_data.set_index('id').apply(lambda row: row['score'] if row['was_home'] else row['opponent_score'], axis=1).to_dict()
opponent_score_mapping = fixtures_2019_20_data.set_index('id').apply(lambda row: row['opponent_score'] if row['was_home'] else row['score'], axis=1).to_dict()
all_player_data.loc[mask_2019_20, 'score'] = all_player_data.loc[mask_2019_20, 'fixture'].map(score_mapping)
all_player_data.loc[mask_2019_20, 'opponent_score'] = all_player_data.loc[mask_2019_20, 'fixture'].map(opponent_score_mapping)

# Checking for missing values in the 2019-20 season data after updating
missing_values_2019_20_after_score_update = all_player_data[mask_2019_20].isnull().sum()
missing_values_2019_20_after_score_update[missing_values_2019_20_after_score_update > 0].sort_values(ascending=False)

position        22560
starts          22560
team_a_score       59
team_h_score       59
dtype: int64

In [27]:
# Imputing the 'position' column with 'UNK' for Unknown
all_player_data['position'].fillna('UNK', inplace=True)

# Make player name consistent across all seasons
all_player_data.loc[all_player_data['season'] == '2019-20', 'name'] = all_player_data['name'].apply(
    lambda x: x.split('_')[0] + " " + x.split('_')[1] if '_' in x else x)

In [28]:
all_player_data

,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,was_home,yellow_cards,gameweek,season,starts,opponent_team_name,difficulty,opponent_difficulty,score,opponent_score
0,Ozan Tufan,MID,Watford,0,0,0,0,0.0,557,268,...,False,0,27,2021-22,NaN,Man Utd,4.0,2.0,0.0,0.0
1,Andre Gray,FWD,Watford,0,0,0,0,0.0,383,268,...,False,0,27,2021-22,NaN,Man Utd,4.0,2.0,0.0,0.0
2,Willy Boly,DEF,Wolves,0,0,0,0,0.0,428,270,...,False,0,27,2021-22,NaN,West Ham,3.0,3.0,0.0,1.0
3,Mads Roerslev Rasmussen,DEF,Brentford,0,0,0,0,0.0,89,262,...,True,0,27,2021-22,NaN,Newcastle,2.0,2.0,0.0,2.0
4,Sam Byram,DEF,Norwich,0,0,0,0,0.0,321,269,...,False,0,27,2021-22,NaN,Southampton,3.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101513,Max Meyer,MID,Crystal Palace,0,0,0,0,0.0,143,222,...,False,0,23,2020-21,NaN,Leeds,3.0,2.0,0.0,2.0
101514,Hakim Ziyech,MID,Chelsea,0,0,0,0,0.0,114,226,...,False,0,23,2020-21,NaN,Sheffield Utd,2.0,4.0,2.0,1.0
101515,Jonny Evans,DEF,Leicester,0,1,25,1,1.3,219,228,...,False,1,23,2020-21,NaN,Wolves,3.0,4.0,0.0,0.0
101516,Kyle Edwards,MID,West Brom,0,0,0,0,0.0,421,227,...,False,0,23,2020-21,NaN,Spurs,4.0,2.0,0.0,2.0


In [29]:
# Sampling the assigned positions for 'Aaron Ramsdale' across all seasons
print(all_player_data[all_player_data['name'] == 'Aaron Ramsdale'][['season', 'position', 'element']].drop_duplicates(), '\n')

def standardize_element_ids(df):
    latest_seasons = df.groupby('name')['season'].transform(max)
    latest_players_df = df[df['season'] == latest_seasons]
    name_to_latest_element = dict(zip(latest_players_df['name'], latest_players_df['element']))
    df['element'] = df['name'].map(name_to_latest_element)
    return df

all_player_data = standardize_element_ids(all_player_data)
print(all_player_data[all_player_data['name'] == 'Aaron Ramsdale'][['season', 'position', 'element']].drop_duplicates())

        season position element
1477   2021-22       GK     559
7772   2021-22      GKP     559
25792  2022-23       GK      15
51956  2019-20      UNK     494
74713  2023-24       GK      17
77903  2020-21       GK     483 

        season position  element
1477   2021-22       GK       17
7772   2021-22      GKP       17
25792  2022-23       GK       17
51956  2019-20      UNK       17
74713  2023-24       GK       17
77903  2020-21       GK       17


In [31]:
# Updating the position for 'Aaron Ramsdale' to be 'GK' consistently across all seasons
all_player_data.loc[all_player_data['name'] == 'Aaron Ramsdale', 'position'] = 'GK'
# Verifying the update by extracting the assigned positions for 'Aaron Ramsdale' across all seasons
ramsdale_positions_updated = all_player_data[all_player_data['name'] == 'Aaron Ramsdale'][['season', 'position']].drop_duplicates()

ramsdale_positions_updated

,season,position
1477,2021-22,GK
25792,2022-23,GK
51956,2019-20,GK
74713,2023-24,GK
77903,2020-21,GK


In [32]:
# Updating all 'GKP' positions to 'GK' for consistency
all_player_data.loc[all_player_data['position'] == 'GKP', 'position'] = 'GK'

# Verifying the update by checking the presence of 'GKP' in the dataset
gkp_exists = 'GKP' in all_player_data['position'].unique()
gkp_exists

False

In [33]:
# Assign position to all UNK positions if they played for more than one season
# Identifying players who played in the 2019-20 season
players_2019_20 = set(all_player_data[all_player_data['season'] == '2019-20']['name'].unique())
players_other_seasons = set(all_player_data[all_player_data['season'] != '2019-20']['name'].unique())
players_both_seasons = players_2019_20.intersection(players_other_seasons)
# Extracting the latest position for the players who played in both the 2019-20 season and at least one other season
latest_positions = all_player_data[all_player_data['name'].isin(players_both_seasons)].groupby('name')['position'].last().to_dict()
all_player_data.loc[(all_player_data['name'].isin(players_both_seasons) & (all_player_data['season'] == '2019-20')), 'position'] = all_player_data['name'].map(latest_positions)


In [34]:
# Drop all players with the UNK position since they do not appear in multiple seasons and are not currently playing either
all_player_data = all_player_data[all_player_data['position'] != 'UNK']

In [35]:
all_player_data

,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,was_home,yellow_cards,gameweek,season,starts,opponent_team_name,difficulty,opponent_difficulty,score,opponent_score
0,Ozan Tufan,MID,Watford,0,0,0,0,0.0,557,268,...,False,0,27,2021-22,NaN,Man Utd,4.0,2.0,0.0,0.0
1,Andre Gray,FWD,Watford,0,0,0,0,0.0,383,268,...,False,0,27,2021-22,NaN,Man Utd,4.0,2.0,0.0,0.0
2,Willy Boly,DEF,Wolves,0,0,0,0,0.0,439,270,...,False,0,27,2021-22,NaN,West Ham,3.0,3.0,0.0,1.0
3,Mads Roerslev Rasmussen,DEF,Brentford,0,0,0,0,0.0,114,262,...,True,0,27,2021-22,NaN,Newcastle,2.0,2.0,0.0,2.0
4,Sam Byram,DEF,Norwich,0,0,0,0,0.0,321,269,...,False,0,27,2021-22,NaN,Southampton,3.0,2.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101513,Max Meyer,MID,Crystal Palace,0,0,0,0,0.0,143,222,...,False,0,23,2020-21,NaN,Leeds,3.0,2.0,0.0,2.0
101514,Hakim Ziyech,MID,Chelsea,0,0,0,0,0.0,218,226,...,False,0,23,2020-21,NaN,Sheffield Utd,2.0,4.0,2.0,1.0
101515,Jonny Evans,DEF,Leicester,0,1,25,1,1.3,703,228,...,False,1,23,2020-21,NaN,Wolves,3.0,4.0,0.0,0.0
101516,Kyle Edwards,MID,West Brom,0,0,0,0,0.0,421,227,...,False,0,23,2020-21,NaN,Spurs,4.0,2.0,0.0,2.0


In [36]:
# Create functions to get this final product of data from all data, modular functions for data transformations so it can be reused
#all_player_data.to_csv('all_fixtures.csv')